In [1]:
print('hello world')

hello world


*[
  _type == "media" &&
  type in ["movie", "show"]
]{
  "title": title,
  "EndedAt": *[
    _type == "session" &&
    media._ref == ^._id &&
    startedAt >= "2025-01-01T00:00:00Z" &&
    startedAt <  "2026-01-01T00:00:00Z"
  ]
  | order(startedAt desc)[0].startedAt
}
[defined(EndedAt)]
| order(EndedAt desc)


In [5]:
pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.0/207.0 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 2.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 25.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 19.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import os
import pandas as pd
from urllib.parse import quote


In [18]:
import os
os.listdir(".")

['studio',
 'web',
 'wrapped.ipynb',
 '.editorconfig',
 'README.md',
 '.gitignore',
 'package-lock.json',
 'package.json',
 'project_secrets.py',
 'lerna.json',
 '.git']

In [2]:
from project_secrets import READ_TOKEN

In [12]:
# --- CONFIG ---
PROJECT_ID = "z417aens"
DATASET = "production"

# GROQ query
groq = """
*[
  _type == "media" &&
  type in ["movie", "show"]
]{
  "title": title,
  "EndedAt": *[
    _type == "session" &&
    media._ref == ^._id &&
    startedAt >= "2025-01-01T00:00:00Z" &&
    startedAt <  "2026-01-01T00:00:00Z"
  ] | order(startedAt desc)[0].startedAt
}
[defined(EndedAt)]
| order(EndedAt desc)
"""

groq = """
*[
  _type == "media" &&
  type in ["movie", "show"]
]{
  "title": title,
  "EndedAt": *[
    _type == "session" &&
    media._ref == ^._id &&
    startedAt >= "2025-01-01T00:00:00Z" &&
    startedAt <  "2026-01-01T00:00:00Z"
  ] | order(startedAt desc)[0].startedAt
}
[defined(EndedAt)]
| order(EndedAt desc)
"""

url = f"https://{PROJECT_ID}.api.sanity.io/v2021-03-25/data/query/{DATASET}?query={quote(groq)}"

headers = {"Authorization": f"Bearer {READ_TOKEN}"} if READ_TOKEN else {}

response = requests.get(url, headers=headers)

# Helpful debug if it still fails
if not response.ok:
    print("Status:", response.status_code)
    print("Body:", response.text)
    response.raise_for_status()

data = response.json()["result"]

df = pd.DataFrame(
    {"datetime": [item["EndedAt"] for item in data],
     "title":    [item["title"]   for item in data]}
)

df.head()


,datetime,title
0,2025-12-05T02:39:06.198Z,Fleabag
1,2025-12-05T02:39:00.000Z,Friday Night Lights
2,2025-11-30T02:39:00.000Z,Sentimental Value
3,2025-11-27T04:10:00.000Z,The Master
4,2025-11-20T02:39:00.000Z,In Treatment
